 Install Dependencies

In [1]:
!pip install -q ultralytics filterpy opencv-python-headless matplotlib pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.0 MB/s eta 0:00:00


Upload Videos and YOLOv11 Model

In [2]:
from google.colab import files

# Upload best.pt, tacticam.mp4, broadcast.mp4
uploaded = files.upload()

Saving tacticam.mp4 to tacticam.mp4
Saving broadcast.mp4 to broadcast.mp4
Saving best.pt to best.pt


Import Libraries

In [23]:
!pip install -q ultralytics filterpy opencv-python-headless lap
!git clone https://github.com/nwojke/deep_sort.git
!git clone https://github.com/ZQPei/deep_sort_pytorch.git
%cd deep_sort_pytorch
!pip install -q -r requirements.txt
%cd ..

Cloning into 'deep_sort'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 167 (delta 7), reused 7 (delta 7), pack-reused 144 (from 2)
Receiving objects: 100% (167/167), 85.78 KiB | 714.00 KiB/s, done.
Resolving deltas: 100% (85/85), done.
Cloning into 'deep_sort_pytorch'...
remote: Enumerating objects: 1078, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 1078 (delta 57), reused 45 (delta 45), pack-reused 979 (from 2)
Receiving objects: 100% (1078/1078), 29.15 MiB | 23.09 MiB/s, done.
Resolving deltas: 100% (507/507), done.
/content/deep_sort_pytorch
ERROR: Invalid requirement: 'torch~=1.12.1+cu113': Local version label can only be used with `==` or `!=` operators
    torch~=1.12.1+cu113
         ~~~~~~~~^ (from line 37 of requirements.txt)
/content


In [39]:
!pip install deep-sort-realtime ultralytics opencv-python-headless filterpy lap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 62.8 MB/s eta 0:00:00


In [40]:
# Import Required Libraries

import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from pathlib import Path
from deep_sort_realtime.deepsort_tracker import DeepSort
from scipy.spatial.distance import cosine

Load YOLO Model

In [41]:
model = YOLO("best.pt")  # Replace with correct path if needed

Detection Function YOLO

In [70]:
def get_yolo_detections(frame, model):
    results = model(frame)[0]
    detections = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        w, h = x2 - x1, y2 - y1
        detections.append(([x1, y1, w, h], conf, cls))
    return detections

DeepSORT + YOLO Tracking Function

In [71]:
def track_video(video_path, camera_name):
    cap = cv2.VideoCapture(video_path)
    deepsort = DeepSort(max_age=30)
    frame_id = 0
    all_tracks = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        detections = get_yolo_detections(frame, model)
        tracks = deepsort.update_tracks(detections, frame=frame)

        for track in tracks:
            if not track.is_confirmed():
                continue
            l, t, r, b = track.to_ltrb()
            tid = track.track_id
            all_tracks.append([frame_id, camera_name, tid, int(l), int(t), int(r), int(b)])

        frame_id += 1

    cap.release()
    df = pd.DataFrame(all_tracks, columns=["frame", "camera", "player_id", "x1", "y1", "x2", "y2"])
    return df

Track Players in Both Videos

In [72]:
df_broadcast = track_video("broadcast.mp4", "broadcast")
df_tacticam = track_video("tacticam.mp4", "tacticam")


0: 384x640 3 players, 72.6ms
Speed: 3.8ms preprocess, 72.6ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 67.4ms
Speed: 3.2ms preprocess, 67.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 43.7ms
Speed: 3.0ms preprocess, 43.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 42.9ms
Speed: 4.0ms preprocess, 42.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 40.7ms
Speed: 3.6ms preprocess, 40.7ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 1 referee, 34.0ms
Speed: 3.0ms preprocess, 34.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalkeeper, 12 players, 1 referee, 33.6ms
Speed: 3.0ms preprocess, 33.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 1 goalkeeper, 14 players, 1 referee, 35.4ms
Speed: 2.

Extract Color Histograms

In [73]:
def extract_color_histograms(df, video_path):
    cap = cv2.VideoCapture(video_path)
    histograms = {}
    grouped = df.groupby("frame")

    for frame_id, frame_data in grouped:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if not ret:
            continue

        for _, row in frame_data.iterrows():
            pid = row.player_id
            x1, y1, x2, y2 = map(int, [row.x1, row.y1, row.x2, row.y2])
            crop = frame[y1:y2, x1:x2]
            if crop.size == 0:
                continue
            hist = cv2.calcHist([crop], [0, 1, 2], None, [8, 8, 8],
                                [0, 256, 0, 256, 0, 256])
            hist = cv2.normalize(hist, hist).flatten()
            histograms[pid] = hist

    cap.release()
    return histograms

hist_broadcast = extract_color_histograms(df_broadcast, "broadcast.mp4")
hist_tacticam = extract_color_histograms(df_tacticam, "tacticam.mp4")

Match Players Across Cameras

In [74]:
def match_players(hist_src, hist_tgt):
    matched_ids = {}
    for tgt_id, tgt_hist in hist_tgt.items():
        best_match = None
        best_score = float("inf")
        for src_id, src_hist in hist_src.items():
            score = cosine(tgt_hist, src_hist)
            if score < best_score:
                best_score = score
                best_match = src_id
        matched_ids[tgt_id] = best_match
    return matched_ids

id_mapping = match_players(hist_broadcast, hist_tacticam)
df_tacticam["player_id"] = df_tacticam["player_id"].map(id_mapping).fillna(-1).astype(int)

Merge & Save Results

In [75]:
df_combined = pd.concat([df_broadcast, df_tacticam])
df_combined.to_csv("final_player_ids.csv", index=False)

 Draw and Save Annotated Videos

In [76]:
def draw_and_save_video(video_path, df, output_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    grouped = df.groupby("frame")
    frame_id = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_id in grouped.groups:
            for _, row in grouped.get_group(frame_id).iterrows():
                x1, y1, x2, y2 = map(int, [row.x1, row.y1, row.x2, row.y2])
                pid = int(row.player_id)
                color = (0, 255, 0) if row.camera == "tacticam" else (255, 0, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"ID: {pid}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        out.write(frame)
        frame_id += 1

    cap.release()
    out.release()

Save Final Output Videos

In [77]:
draw_and_save_video("broadcast.mp4", df_broadcast, "output_broadcast.mp4")
draw_and_save_video("tacticam.mp4", df_tacticam, "output_tacticam.mp4")